## Use parallelism in notebook

In [84]:
import ipyparallel as ipp
c = ipp.Client(profile='default')

In [86]:
view = c[:]
view.activate()
view.block = True

## Initialisation process

In [87]:
%%px
from mpi4py import MPI
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as linsolve

size = MPI.COMM_WORLD.size
rank = MPI.COMM_WORLD.rank

print("Size: {}, Rank: {}".format(size, rank))

N = 101
L = 1.
dx = 1./(N - 1)

[stdout:0] Size: 4, Rank: 0
[stdout:1] Size: 4, Rank: 1
[stdout:2] Size: 4, Rank: 2
[stdout:3] Size: 4, Rank: 3


## Division of domain into subdomains

In [88]:
%%px
Nl = (N-1)//size+1
if MPI.COMM_WORLD.rank == size-1:
    Nl += (N-1)%size
    
Narray = MPI.COMM_WORLD.allgather(Nl-1)
tmp = np.cumsum(Narray)
tmp = np.insert(tmp, 0, 0)
tmp

Out[0:2]: array([  0,  25,  50,  75, 100])

Out[1:2]: array([  0,  25,  50,  75, 100])

Out[2:2]: array([  0,  25,  50,  75, 100])

Out[3:2]: array([  0,  25,  50,  75, 100])

In [89]:
%%px

beg = tmp[rank]*dx
end = tmp[rank+1]*dx

overlap = 2

if rank > 0:
    beg -= dx * overlap
if rank < size-1:
    end += dx * overlap
    
xl = np.arange(beg, end + 0.5*dx, dx)

## Construction of local matrices

In [90]:
%%px
n = xl.size

A = sp.spdiags([-np.ones(n), 2*np.ones(n), -np.ones(n)], [-1, 0, 1], n, n)
A = A.tocsr()

A[0, 0] = 1; A[0, 1] = 0;
A[-1, -2] = 0; A[-1, -1] = 1;

b = np.ones(n)*dx**2
b[0] = 0; b[-1] = 0

## Solve the local problems

In [91]:
%%px
LU = linsolve.factorized(A.tocsc())
u = LU(b)

## Alternating Schwarz algorithm

<table>
<tr>
<td>
$$
\scriptsize{
\left\{
\begin{array}{l}
\mathcal{L}(u_1^{k+1}) = f(x), \; x \in \Omega_1 \\
u_1^{k+1}(x) = g(x), \; x\in \partial \Omega_1 \bigcap \overline \Omega_1 \\
u_1^{k+1}(x) = u_2^k(x), \; x \in \Gamma_1
\end{array}
\right.}
$$
    </td>
<td>
$$
\scriptsize{
\left\{
\begin{array}{l}
\mathcal{L}(u_2^{k+1}) = f(x), \; x \in \Omega_2 \\
u_2^{k+1}(x) = g(x), \; x\in \partial \Omega_2 \bigcap \overline \Omega_2 \\
u_2^{k+1}(x) = u_1^{k+1}(x), \; x\in \Gamma_2
\end{array}
\right.
}
$$
    </td>
    </tr>
</table>

## Parallel Schwarz algorithm

<table>
<tr>
<td>
$$
\scriptsize{
\left\{
\begin{array}{l}
\mathcal{L}(u_1^{k+1}) = f(x), \; x \in \Omega_1 \\
u_1^{k+1}(x) = g(x), \; x\in \partial \Omega_1 \bigcap \overline \Omega_1 \\
u_1^{k+1}(x) = u_2^k(x), \; x \in \Gamma_1
\end{array}
\right.}
$$
    </td>
<td>
$$
\scriptsize{
\left\{
\begin{array}{l}
\mathcal{L}(u_2^{k+1}) = f(x), \; x \in \Omega_2 \\
u_2^{k+1}(x) = g(x), \; x\in \partial \Omega_2 \bigcap \overline \Omega_2 \\
u_2^{k+1}(x) = u_1^{k}(x), \; x\in \Gamma_2
\end{array}
\right.
}
$$
    </td>
    </tr>
</table>

In [92]:
%%px
def update(nbite=100):
    U = []
    U.append(MPI.COMM_WORLD.allgather(u))
    X = MPI.COMM_WORLD.allgather(xl)
    for k in range(nbite):
        if rank == 0:
            MPI.COMM_WORLD.send(u[-1-2*overlap], rank+1, rank)
            b[-1] = MPI.COMM_WORLD.recv(None, rank+1, rank+1)
        elif rank == size-1:
            MPI.COMM_WORLD.send(u[2*overlap], rank-1, rank)
            b[0] = MPI.COMM_WORLD.recv(None, rank-1, rank-1)
        else:
            MPI.COMM_WORLD.send(u[-1-2*overlap], rank+1, rank)
            b[-1] = MPI.COMM_WORLD.recv(None, rank+1, rank+1)
            MPI.COMM_WORLD.send(u[2*overlap], rank-1, rank)
            b[0] = MPI.COMM_WORLD.recv(None, rank-1, rank-1)
        u[:] = LU(b)
        U.append(MPI.COMM_WORLD.allgather(u))
    return X, U

In [100]:
%%px
X, U = update(50)

KeyboardInterrupt: 

In [97]:
%%px
def post_process(X, U):
    x_ = np.array([])
    u_ = np.array([])
    for n in range(size):
        if n < size-1:
            x_ = np.hstack((x_, X[n][:-1-2*overlap]))
            u_ = np.hstack((u_, U[-1][n][:-1-2*overlap]))
        else:
            x_ = np.hstack((x_, X[n][:]))
            u_ = np.hstack((u_, U[-1][n][:]))
    return x_, u_


x_, u_ = post_process(X, U)

In [98]:
%%px
if rank == 0:
    import matplotlib.pyplot as plt
    plt.plot(x_, u_)
    plt.show()

KeyboardInterrupt: 